In [14]:
! pip install langchain langchain_openai langchain_community python-dotenv pymysql PyAthena 


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [15]:
from text2sql import Text2Sql

agent = Text2Sql()
questions = [
    "What are the top 10 highest-rated titles with more than 1,000 votes",
    "What is the average rating for titles that have votes between 500 and 5,000?",
    "For each unique rating, what is the title with the highest number of votes?",
    "What is the average number of votes for titles based on their rating rounded off to the nearest whole number?",
    "How many titles have a rating above the median rating value?",
    "Which movie had the highest rating?",
]

In [16]:
conn = agent._establish_athena_conn()
print(conn.get_table_names())
print("\n")
print(conn.get_table_info())

['ratings']


2024-06-05 02:44:59,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-05 02:44:59,527 INFO sqlalchemy.engine.Engine SELECT ratings.tconst, ratings.averagerating, ratings.numvotes 
FROM ratings LIMIT %(param_1)s
2024-06-05 02:44:59,527 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {'param_1': 3}


/Users/camanze/Desktop/aws/hack2sql/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 0.3.0. Use get_usable_table_names instead.
  warn_deprecated(


2024-06-05 02:45:00,857 INFO sqlalchemy.engine.Engine ROLLBACK

CREATE EXTERNAL TABLE ratings (
	tconst STRING,
	averagerating FLOAT,
	numvotes BIGINT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'field.delim' = '	'
)
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://fsptext2sql/ratings/'
TBLPROPERTIES (
	'CrawlerSchemaDeserializerVersion' = '1.0',
	'CrawlerSchemaSerializerVersion' = '1.0',
	'UPDATED_BY_CRAWLER' = 'ratings_crawler',
	'areColumnsQuoted' = 'false',
	'averageRecordSize' = '21',
	'classification' = 'csv',
	'columnsOrdered' = 'true',
	'compressionType' = 'none',
	'delimiter' = '	',
	'inputformat' = 'org.apache.hadoop.mapred.TextInputFormat',
	'location' = 's3://fsptext2sql/ratings/',
	'objectCount' = '1',
	'outputformat' = 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
	'recordCount' = '1193012',
	'serde.se

In [17]:
sql_chain = agent.create_sql_chain()
sql_chain.get_prompts()[0].pretty_print()

Given an input question, first create a syntactically correct awsathena query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most 5 results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}


In [18]:
sql_chain.invoke({"question": questions[0]})

2024-06-05 02:45:00,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-05 02:45:00,894 INFO sqlalchemy.engine.Engine SELECT ratings.tconst, ratings.averagerating, ratings.numvotes 
FROM ratings LIMIT %(param_1)s
2024-06-05 02:45:00,894 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {'param_1': 3}
2024-06-05 02:45:02,230 INFO sqlalchemy.engine.Engine ROLLBACK
2024-06-05 02:45:04,954 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-05 02:45:04,955 INFO sqlalchemy.engine.Engine SELECT tconst, averagerating FROM ratings WHERE numvotes > 1000 ORDER BY averagerating DESC LIMIT 10
2024-06-05 02:45:04,955 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {}
2024-06-05 02:45:07,350 INFO sqlalchemy.engine.Engine COMMIT
[('tt2301451', 10.0), ('tt9892336', 10.0), ('tt30882773', 10.0), ('tt32368060', 10.0), ('tt1204265', 9.9), ('tt11294360', 9.9), ('tt0701989', 9.9), ('tt11028174', 9.9), ('tt12187040', 9.9), ('tt2178784', 9.9)]


In [21]:
chain = agent.rephrase_chain()
chain.invoke({"question" : questions[4]})

2024-06-05 02:45:56,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-05 02:45:56,235 INFO sqlalchemy.engine.Engine SELECT ratings.tconst, ratings.averagerating, ratings.numvotes 
FROM ratings LIMIT %(param_1)s
2024-06-05 02:45:56,235 INFO sqlalchemy.engine.Engine [cached since 55.34s ago] {'param_1': 3}
2024-06-05 02:45:57,563 INFO sqlalchemy.engine.Engine ROLLBACK
2024-06-05 02:45:59,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-05 02:45:59,996 INFO sqlalchemy.engine.Engine SELECT COUNT(tconst) 
FROM ratings 
WHERE averagerating > (SELECT AVG(averagerating) FROM ratings)
2024-06-05 02:45:59,997 INFO sqlalchemy.engine.Engine [cached since 20.28s ago] {}
2024-06-05 02:46:03,433 INFO sqlalchemy.engine.Engine COMMIT


'815,853 titles have a rating above the median rating value.'